In [ ]:
from sklearn.cross_validation import KFold
from sklearn.datasets import fetch_mldata
from sklearn import linear_model, datasets
import numpy as np
import os
from pylab import *
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

mnist = fetch_mldata('MNIST original')
#mnist = datasets.load_iris()


matrix_data, y_data = mnist.data / 255., mnist.target

matrix = np.concatenate((matrix_data[y_data==0], matrix_data[y_data==1], matrix_data[y_data==2]), axis=0)
y = np.concatenate((y_data[y_data==0], y_data[y_data==1], y_data[y_data==2]), axis=0)


def findSoftMaxFunction(X, y, className, theta_dict):
    
    upper_exp = (np.asarray(np.exp(theta_dict[className] * np.matrix(X).T)))[0]
    lower_exp = np.zeros(len(upper_exp))
    for i in range(len(theta_dict)):
        lower_exp = lower_exp + (np.asarray(np.exp(theta_dict[i] * np.matrix(X).T)))[0]
            
    softmax = upper_exp/lower_exp
    return softmax

def findTheta(xX, xy, xclassName, xtheta_dict, xsoftmax_fun):
    
    xclassX = xX[xy==xclassName]
    xy_original = xy
    
    """for i in range(len(xy_original)):
        if(int(xy_original[i])!=xclassName):
            xy[i] == 0
        else: 
            xy[i] == 1
"""
    #for i in range(len(xsoftmax_fun)):
    #xtheta_dict[xclassName] = xtheta_dict[xclassName] - 0.01 * ((xsoftmax_fun[i][xy==xclassName]-xy[i][xy==xclassName]) * xX[i][xy==xclassName])
        
    h_theta = xsoftmax_fun
    h_theta = h_theta.reshape(h_theta.shape[0], 1)
    xy = xy.reshape(xy.shape[0], 1)
    
    xtheta_dict[xclassName] = xtheta_dict[xclassName] - 0.0001 * np.sum(((h_theta[xy_original==xclassName]-xy[xy_original==xclassName]) * xX[xy_original==xclassName]), axis=0)
    
    #print xtheta_dict
    return xtheta_dict        

def discriminant_function(max_h_theta, clas):
        
    diff = []
    predicted = []
    array = []

    for index in range(len(clas)):
        array.append(max_h_theta[index])
    predicted_values = np.maximum.reduce(array)

    for x in range(len(predicted_values)):
        for y in max_h_theta:
            value = max_h_theta[y]
            if(value[x] == predicted_values[x]):
                predicted.append(y)
    #print predicted
    return predicted


def findOtherParameters(confusion_mat):
    
    list_diagonal = np.zeros(confusion_mat.shape[0])
    list_row_sum = np.zeros(confusion_mat.shape[0])
    list_column_sum=np.zeros(confusion_mat.shape[1])
    
    precision_value = []
    recall_value = []
    f_measure_value = []
    
    total = np.sum(confusion_mat)
    confuse_diagonal = 0

    for i in range(confusion_mat.shape[0]):
        for j in range(confusion_mat.shape[1]):
            list_row_sum[i] += confusion_mat[i][j]
            list_column_sum[i] += confusion_mat[j][i]
            if(i==j):
                list_diagonal[i] = confusion_mat[i][j]
                confuse_diagonal +=  confusion_mat[i][j]
    
    accuracy = float(confuse_diagonal)/total

    for index in range(len(list_row_sum)):
        if list_row_sum[index]==0:
            precision_value.append(0.0)
        else:
            precision_value.append((float)(list_diagonal[index]) / list_row_sum[index])
            
        if list_column_sum[index]==0:
            recall_value.append(0)
        else:
            recall_value.append((float)(list_diagonal[index]) / list_column_sum[index])
        
        if precision_value[index]==0 or recall_value[index]==0:
            f_measure_value.append(0)
        else:
            f_measure_value.append((float) (2 * precision_value[index] * recall_value[index]) / (precision_value[index] + recall_value[index]))
        
    return accuracy, precision_value, recall_value, f_measure_value
        

i=0
max_gX = {}
theta_dictionary = {}

kf = KFold(matrix.shape[0], n_folds=10, shuffle=True)

for train_index, test_index in kf:
     
    #print "train index=", train_index
    #print "test index=", test_index
    
    X_train, X_test = matrix[train_index], matrix[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    size=len(y_train)
    
    for clas in np.unique(y_train):
        for clas_in in np.unique(y_train):
            theta_dictionary.update({int(clas_in):np.zeros(np.array(X_train.shape[1]))})
        for i in range(2):
            softmax = findSoftMaxFunction(X_train, y_train, clas, theta_dictionary)
            theta_dictionary = findTheta(X_train, y_train, clas, theta_dictionary, softmax)
            
        max_gX.update({int(clas):theta_dictionary[clas]})
    
    max_h_value={}
    for cl in np.unique(y_train):
        h_value = np.matrix(max_gX[cl]) * np.matrix(X_test).T
        max_h_value.update({int(cl): np.asarray(h_value)[0]})

#    print "dict=", max_h_value
    predictY = discriminant_function(max_h_value, np.unique(y_train))
    
    print accuracy_score(y_test,np.array(predictY))
    
    count=0
    for i in range(len(y_test)):
        if y_test[i]==predictY[i]:
            count +=1
    
    confusion_mat = confusion_matrix(y_test, predictY)
    accuracy, precision_val, recall_val, f_measure_val = findOtherParameters(confusion_mat)
    
    print "Final Output : "
    print "Accuracy = ", accuracy*100, " % "
    print "Precision = ", precision_val
    print "Recall = ", recall_val
    print "F-Measure", f_measure_val
    print "\n Fold Completed \n"
    
    break
    